[How to use a model to call tools](https://python.langchain.com/v0.2/docs/how_to/tool_calling/)

In [1]:
import getpass
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass()
os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=os.environ['OPENAI_API_KEY'], 
    openai_api_base='https://api.deepseek.com'
)

Sadly, our DeepSeek model is not the same as OpenAI in tool calling.

Let's dive into the nature of tool calling. See [here](https://python.langchain.com/v0.1/docs/use_cases/tool_use/prompting/).

In [5]:
# Tool should be a normal function, with params and returns
# Here we define a mock tool, in the real world, it may be a search-engine api or something else
from langchain_core.tools import tool

@tool
def negative_comment(game: str) -> str:
  '''Give negative comment to this game.'''
  return f'{game} seems not better than Genshin Impact...'

print(negative_comment.name)
print(negative_comment.description)
print(negative_comment.args)
negative_comment.invoke('Zelda')


negative_comment
Give negative comment to this game.
{'game': {'title': 'Game', 'type': 'string'}}


'Zelda seems not better than Genshin Impact...'

In [7]:
# We should tell LLM about the tool's info, which means that
# LLM will choose tool to use according to its description (and may also args)
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([negative_comment])
rendered_tools

'negative_comment(game: str) -> str - Give negative comment to this game.'

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f'''You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.'''

prompt = ChatPromptTemplate.from_messages(
    [('system', system_prompt), ('user', 'Give negative comment to {input}')]
)

In [10]:
# Through prompt, we transform natural language instruction => formatted tool call
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({'input': 'Zelda'})

{'name': 'negative_comment', 'arguments': {'game': 'Zelda'}}

In [11]:
# Then we directly do tool calling
from operator import itemgetter

chain = prompt | model | JsonOutputParser() | itemgetter('arguments') | negative_comment
chain.invoke({'input': 'Zelda'})

'Zelda seems not better than Genshin Impact...'

In [20]:
# If we have multiple tools to choose, how to do decision?
# According to input, the LLM will choose one tool, we use tool.name to locate
@tool
def positive_comment(game: str) -> str:
  '''Give positive comment to this game.'''
  return f'{game} is the best game in the world !!!'

tools = [negative_comment, positive_comment]

def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output['name']]
    return itemgetter('arguments') | chosen_tool


rendered_tools = render_text_description([negative_comment, positive_comment])
system_prompt = f'''You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.'''

prompt = ChatPromptTemplate.from_messages(
    [('system', system_prompt), ('user', 'If {input} belongs to Zelda series, give positive comment. Otherwise give negative comment.')]
)

chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input": "Zelda"})

'Zelda is the best game in the world !!!'

In [21]:
chain.invoke({"input": "DotA2"})

'DotA2 seems not better than Genshin Impact...'

In [22]:
# Also return tool input
from langchain_core.runnables import RunnablePassthrough

chain = prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)
chain.invoke({"input": "Zelda"})

{'name': 'positive_comment',
 'arguments': {'game': 'Zelda'},
 'output': 'Zelda is the best game in the world !!!'}